<a href="https://colab.research.google.com/github/dude123studios/AdvancedDeepLearning/blob/main/Colorization_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import packages

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, LeakyReLU, ReLU, Conv2D, UpSampling2D, MaxPooling2D, Activation,BatchNormalization, Dropout,Input,Concatenate,Flatten,Reshape
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import cv2

Define a method to convert real images to gray images

In [2]:
def get_gray_images(colored_images):
  #                       blue                              green                            red
  grayValues = 0.07 * colored_images[:,:,:,2] + 0.72 * colored_images[:,:,:,1] + 0.21 * colored_images[:,:,:,0]
  gray_images = (grayValues/225).astype(np.float32)
  gray_images = gray_images.reshape((gray_images.shape[0],gray_images.shape[1],gray_images.shape[2],1))
  return gray_images

Define a method to load our dataset and get gray images

In [3]:
def load_dataset():
  (x_train,_),(x_test,_) = cifar10.load_data()
  return (x_train,get_gray_images(x_train)), (x_test,get_gray_images(x_test))

Define a downsample block

In [4]:
def downsample(filters,dropout=False,pool=True):
  model = tf.keras.models.Sequential()
  model.add(Conv2D(filters,(3,3),padding='same'))
  model.add(BatchNormalization())
  model.add(ReLU())
  if pool:
    model.add(MaxPooling2D(2,2))
  if dropout:
    model.add(Dropout(0.3))
  return model  

Define an upsample block

In [5]:
def upsample(filters,up=True):
  model = tf.keras.models.Sequential()
  model.add(Conv2D(filters,(3,3),padding='same'))
  model.add(BatchNormalization())
  model.add(ReLU())
  if up:
    model.add(UpSampling2D())
  return model

Define Latent Space

In [6]:
def latent_space():
  model = tf.keras.models.Sequential()
  model.add(Flatten())
  model.add(Dense(512,input_dim=(1024,),activation='relu'))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(512,activation='relu'))
  model.add(Reshape((2,2,128)))
  return model

Finally, define the entire auto encoder similar to a U-net

In [7]:
def AutoEncoder():
  inputs = Input(shape=[32,32,1])
  skips = []
  x = downsample(32)(inputs)
  skips.append(x)
  x = downsample(64,dropout=True)(x)
  skips.append(x)
  x = downsample(128)(x)
  skips.append(x)
  x = downsample(256,dropout=True)(x)

  x = latent_space()(x)
  
  x = upsample(256)(x)
  concat1 = Concatenate()
  x = upsample(128)(concat1([x,skips[2]]))
  concat2 = Concatenate()
  x = upsample(64)(concat2([x,skips[1]]))
  concat3 = Concatenate()
  x = upsample(32)(concat3([x,skips[0]]))

  x = Conv2D(3, (3,3),padding='same',activation='relu')(x)

  return tf.keras.models.Model(inputs,x)

Define a method to compile our model

In [8]:
def compile(model):
  optimizer = Adam(learning_rate=0.001)
  model.compile(loss='mse',optimizer=optimizer,metrics=[tf.keras.metrics.MeanSquaredError()])

Define a method to train with a GPU

In [9]:
def train(model,gray,real,gray_test,real_test,epochs=50,batch_size=128):
  with tf.device('gpu:0'):
    history = model.fit(gray,real,epochs=epochs,batch_size=batch_size,validation_data=(gray_test,real_test))

Finally, train our model

In [10]:
(real,gray),(real_test,gray_test) = load_dataset()
model = AutoEncoder()
model.summary()
compile(model)
train(model,gray,real,gray_test,real_test,epochs=50)
model.save('colorization_autoencoder.h5')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 16, 16, 32)   448         input_1[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 8, 8, 64)     18752       sequential[0][0]                 
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 4, 4, 128)    74368       sequential_1[0][0]               
______________________________________________________________________________________________

**Colorization** is a hard task, and I'm sure with **100**-**200**~ epochs we would converge to an error of about **100**. This is really good, because with a batch size of **128**, the error of an entire image is around **0.25**. You can go to the colab and try it yourself. The biggest bottleneck for this task, however, is the fact that the images are only **32 x 32**. If I used a better dataset, such as the Yosemite dataset, this would work a lot better. However, that is a test for another day. At a sence, colorization is segmantic segmentation, with a color gradient(shading, not computational)